In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
link_df = pd.read_csv('../../data/original/links (table 3).csv', index_col=0)

In [3]:
validation_path = '../../data/online/'
window_date = pd.to_datetime('2016-10-18')

In [4]:
afternoon_predict_dict = {}
morning_predict_dict = {}

afternoon_real_dict = {}
morning_real_dict = {}

for link_id in link_df.index.values:
    predict_afternoon_m1_df = pd.read_csv(validation_path + 'prediction/feature1/afternoon/' + str(link_id) + '.csv', index_col=0)
    predict_afternoon_m2_df = pd.read_csv(validation_path + 'prediction/feature2/afternoon/' + str(link_id) + '.csv', index_col=0)
    
    predict_morning_m1_df = pd.read_csv(validation_path + 'prediction/feature1/morning/' + str(link_id) + '.csv', index_col=0)
    predict_morning_m2_df = pd.read_csv(validation_path + 'prediction/feature2/morning/' + str(link_id) + '.csv', index_col=0)
    
    
    afternoon_predict_dict[link_id] = [predict_afternoon_m1_df, predict_afternoon_m2_df]
    morning_predict_dict[link_id] = [predict_morning_m1_df, predict_morning_m2_df]

In [5]:
def ArrayMape(ground_truth, predictions):
    diff = (ground_truth - predictions) / ground_truth
    diff = np.abs(diff)
    mape = diff.sum() / ground_truth.size
    
    return mape

In [6]:
def ScalarMape(ground_truth, predictions):
    diff = (ground_truth - predictions) / ground_truth
    diff = np.abs(diff)
    mape = diff
    
    return mape

In [7]:
routes = pd.read_csv('../../data/original/routes (table 4).csv')
submissions = []
for i in routes.index:
    link_seq = routes.loc[i]['link_seq']
    links = link_seq.split(',')
    
    #morning
    start_time = window_date + datetime.timedelta(hours=8)
    for d in range(7):
        current_time = start_time
        for t in range(6):
            predict_time = 0
            real_time = 0
            record = []
            record.append(routes.loc[i]['intersection_id'])
            record.append(routes.loc[i]['tollgate_id'])
            record.append('[' + str(current_time) + ',' + str(current_time + datetime.timedelta(minutes=20)) + ')')
            for link_id in links:
                predict_time = predict_time + morning_predict_dict[int(link_id)][0].loc[d][t] + morning_predict_dict[int(link_id)][1].loc[d][t] / 2
                
            record.append(predict_time)

            submissions.append(record)
            current_time = current_time + datetime.timedelta(minutes=20)
            
        start_time = start_time + datetime.timedelta(days=1)
        
    #afternoon
    start_time = window_date + datetime.timedelta(hours=17)

    for d in range(7):
        current_time = start_time
        for t in range(6):
            predict_time = 0
            real_time = 0
            record = []
            record.append(routes.loc[i]['intersection_id'])
            record.append(routes.loc[i]['tollgate_id'])
            record.append('[' + str(current_time) + ',' + str(current_time + datetime.timedelta(minutes=20)) + ')')
            for link_id in links:
                predict_time = predict_time + afternoon_predict_dict[int(link_id)][0].loc[d][t] + afternoon_predict_dict[int(link_id)][1].loc[d][t] / 2
                
            record.append(predict_time)

            submissions.append(record)
            current_time = current_time + datetime.timedelta(minutes=20)
            
        start_time = start_time + datetime.timedelta(days=1)

In [8]:
submission_df = pd.DataFrame(submissions).rename(columns={0:'intersection_id', 1:'tollgate_id', 2:'time_window', 3:'avg_travel_time'})

In [9]:
submission_df.to_csv(validation_path + '/prediction/combine/prediction_m1_m2.csv', index=False)

In [14]:
submission_df

,intersection_id,tollgate_id,time_window,avg_travel_time
0,A,2,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",66.634701
1,A,2,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",70.649504
2,A,2,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",70.993241
3,A,2,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",66.303417
4,A,2,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",67.784202
5,A,2,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",64.678761
6,A,2,"[2016-10-19 08:00:00,2016-10-19 08:20:00)",65.891600
7,A,2,"[2016-10-19 08:20:00,2016-10-19 08:40:00)",70.785648
8,A,2,"[2016-10-19 08:40:00,2016-10-19 09:00:00)",71.328278
9,A,2,"[2016-10-19 09:00:00,2016-10-19 09:20:00)",66.179242
